In [2]:
import tkinter as tk

#Hiển thị thông báo khi xong giải pháp cuối cùng
from tkinter import messagebox
# Thêm queue để xử lý hàng đợi
from queue import Queue


# --- Thuật toán BFS cho 8 con hậu (lấy tất cả nghiệm) ---
def isSafe(vitri, row, col):
    for r, c in enumerate(vitri):
        if c == col or abs(c - col) == abs(r - row):
            return False
    return True


def bfs_8queens_all():
    n = 8
    q = Queue()
    q.put([])
    solutions = []

    while not q.empty():
        vitri = q.get()
        row = len(vitri)
        if row == n:
            solutions.append(vitri)
            continue
        for col in range(n):
            if isSafe(vitri, row, col):
                q.put(vitri + [col])
    return solutions


# ======== Giao diện chính (GUI) ========
root = tk.Tk()
root.title("Bài toán 8 con hậu - BFS")

root.minsize(550, 500)
root.maxsize(2000, 1600)
root.geometry("1200x700+100+30")
root.configure(bg="#BDE7E7")

title_label = tk.Label(root, text="8 QUEENS BY TEE (BFS)",
                       font=("SegoeUI", 20, "bold"),
                       fg="#0E2846", bg="#BDE7E7")
title_label.pack(pady=10)

main_frame = tk.Frame(root, bg="#BDE7E7")
main_frame.pack(fill="both", expand=True, padx=10, pady=10)

# Bàn cờ trống (bên trái - tham khảo)
board = tk.Frame(main_frame, width=480, height=480)
board.pack(side="left", padx=20)
board.pack_propagate(False)

for row in range(8):
    for col in range(8):
        color = "#47C0C0" if (row + col) % 2 == 0 else "#17375C"
        cell = tk.Frame(board, width=60, height=60, bg=color)
        cell.grid(row=row, column=col)


# Khung hiển thị hậu (bên phải - để hiển thị nghiệm)
placedFrame = tk.Frame(main_frame, width=480, height=480)
placedFrame.pack(side="right", padx=20)
placedFrame.pack_propagate(False)


def draw_empty_board():
    """Vẽ bàn cờ trống bên phải"""
    for widget in placedFrame.winfo_children():
        widget.destroy()
    cells = []
    for row in range(8):
        row_cells = []
        for col in range(8):
            color = "#47C0C0" if (row + col) % 2 == 0 else "#17375C"
            cell = tk.Frame(placedFrame, width=60, height=60, bg=color)
            cell.grid(row=row, column=col)
            cell.pack_propagate(False)
            row_cells.append(cell)
        cells.append(row_cells)
    return cells


def draw_solution(cells, solution):
    """Vẽ một nghiệm lên bàn"""
    for row in range(8):
        for col in range(8):
            for widget in cells[row][col].winfo_children():
                widget.destroy()
    for row, col in enumerate(solution):
        lbl = tk.Label(cells[row][col], text="\u2655", fg="red",
                       bg=cells[row][col]["bg"], font=("Arial", 24, "bold"))
        lbl.pack(expand=True, fill="both")


# --- Quản lý nghiệm ---
all_solutions = []
current_index = 0
cells_global = draw_empty_board()


def show_first_solution():
    global all_solutions, current_index, cells_global
    all_solutions = bfs_8queens_all()
    current_index = 0
    cells_global = draw_empty_board()
    if all_solutions:
        draw_solution(cells_global, all_solutions[current_index])
        update_status()


def show_next_solution():
    global current_index, all_solutions, cells_global
    if not all_solutions:
        return
    current_index = (current_index + 1) % len(all_solutions)
    cells_global = draw_empty_board()
    draw_solution(cells_global, all_solutions[current_index])
    update_status()

    if current_index == len(all_solutions) - 1:
        messagebox.showinfo("Thông báo", "Bạn đã tới giải pháp thứ 92 (cuối cùng)!")


def reset_board():
    global all_solutions, current_index, cells_global
    all_solutions = []
    current_index = 0
    cells_global = draw_empty_board()
    status_label.config(text="No solution loaded.")


def update_status():
    status_label.config(
        text=f"Giải pháp {current_index+1}/{len(all_solutions)}"
    )


# --- Bảng điều khiển ---
control_panel = tk.Frame(root, bg="#BDE7E7")
control_panel.pack(pady=5)

btn_place = tk.Button(control_panel, text="\u25B6",  # ▶
                      font=("Arial", 30, "bold"),
                      width=5, command=show_first_solution)
btn_place.pack(side="left", padx=10)

btn_next = tk.Button(control_panel, text="\u23ED",  # ⏭
                     font=("Arial", 30, "bold"),
                     width=5, command=show_next_solution)
btn_next.pack(side="left", padx=10)

btn_reset = tk.Button(control_panel, text="\u21BA",  # ↺
                      font=("Arial", 30, "bold"),
                      width=5, command=reset_board)
btn_reset.pack(side="left", padx=5)

status_label = tk.Label(root, text="Chưa thực hiện giải pháp nào",
                        font=("SegoeUI", 14), bg="#BDE7E7", fg="#0E2846")
status_label.pack(pady=10)

root.mainloop()
